# Resolução de puzzles Sokoban

## Nota sobre este relatório

Foi utilizada a ferramenta Jupyter Notebook, para tirar proveito de algumas funcionalidades do IPython para análise estatística da execução dos algoritmos.

## Formulação

Foi escolhido representar um estado do puzzle sokoban usando a classe `EstadoSokoban` que vai ser descrita nesta capítulo.

### Atributos

A classe `EstadoSokoban` contém os seguintes atributos:

- `tabuleiro`: lista de listas que representa o puzzle. Cada uma das listas interiores vai ter:

    - ’#’   – para representar as paredes;
    
    - ’.’   – para representar as posições livres;
    
    - ’*’   – para representar as caixas;
    
    - ’o’   – (um ó minúsculo) para representar os alvos;
    
    - ’A’   – para representar o arrumador.
    
    - ’@’   – para representar uma caixa numa posição alvo.
    
    - ’B’   – para representar o arrumador em cima de uma posição alvo.
    
- `arrumador`: posição do arrumador, tuplo (x,y).

- `caixas`: posição das caixas, lista de tuplos (x,y).

- `alvos`: posição dos alvos, lista de tuplos (x,y).

### Métodos

Para além da representação, foram implementados os seguintes métodos:

- `pos_livre(self, x, y)`: verifica se uma posição do puzzle está livre, isto é, se o arrumador pode ir para essa posição.

- `pos_caixa(self, x, y)`: verifica se uma posição do puzzle tem uma caixa.

- `ver_*(self, x, y)`: utiliza as funções anteriores para verificar as posições "cima", "baixo", "direita" e "esquerda" (a cada uma destas opções corresponde um método diferente). A função retorna a ação possível. Se a posição na direção desejada estiver livre, retorna WALK_direção. Se estiver uma caixa, e puder empurrar essa caixa (se na posição seguinte na mesma direção estiver livre), retorna PUSH_direção

- __str__(self): retorno visual do estado

- __gt__(self); __eq__(self, estado); __hash__(self, estado): são métodos necessários para a implementação dos algoritmos de procura com grafos
 
### Implementação da classe concreta 'Problem'
A classe 'Sokoban', extensão da classe 'Problem' tem o seguinte atributo:

- 'estado_inicial': estado inicial com que o problema é inicializado.

- 'tabuleiro_inicial': é uma redundância pois o tabuleiro está no estado_inicial, mas assim facilita a execução de alguns métodos.

- 'deadlocks': lista de posições que são "pontos mortos", que vem do método 'deadlocks_tabuleiro(self)'

Para aplicar os algoritmos de procura de espaço de estados, foi usada uma implementação concreta da classe `Problem` que para além dos métodos abstratos `actions()`, `result()`, `path_cost()` e `goal_test()`, foram ainda implementados os seguintes:

- `pos_deadlock_canto`: Deteção de posições de canto, se os cantos não forem posições alvo. Posições em que a caixa perde liberdade nos 2 sentidos. Sabendo que se uma caixa for para uma posição destas não poderá sair de lá, evitamos que o arrumador teste os estados em que coloca as caixas nestas posições, garantindo assim uma maior rapidez e eficiência.
	Exemplo:
		############
		#..........#
		#o...*...A.#
		#..........#
		############
	Pegando no exemplo anterior, esta função retira os cantos, pois não estão lá alvos e são "pontos mortos", ou deadlocks.
	Resultado:
		############
		#_........_#
		#o...*...A.#
		#_........_#
		############
	Assim, representado por "_", apenas para demonstração, estão as posições que serão acrescentadas à lista de deadlocks de tabuleiro que a classe Sokoban(Problem) tem.

- `deadlock_parede`: Deteção das posições em que a caixa vai perder liberdade em 1 dos sentidos. Se ficar encostada a uma parede, e não houver um alvo nessa linha, ou coluna (paralela à parede), a caixa nunca poderá chegar ao alvo. Pegando no exemplo anterior:
	Exemplo anterior:
		############
		#_........_#
		#o...*...A.#
		#_........_#
		############
	O que este método faz é, com base nos cantos detectados no método anterior, este método verifica se 2 cantos estão alinhados, quer na vertical ou na horizontal, e depois faz um varrimento para ver se é tudo parede entre esses 2 cantos ou não, e se não está em cruzamento com alvos. Serão detectadas mais posições, como mostramos a seguir:	

		############
		#_%%%%%%%%_#
		#o...*...A.#
		#_%%%%%%%%_#
		############
	Representado por "%", apenas para demonstração, 
- `deadlocks_tabuleiro`: Este método usa os dois métodos anteriores para retornar uma lista completa de posições de deadlocks.


## Heurísticas definidas

### Distância euclidiana do arrumador à caixa mais próxima

método: `heur_euclidean_usher_target(nodo)`

A primeira heuristica programada foi a distância euclidiana do arrumador ao alvo mais próximo. Esta heuristica permite também aproximar o arrumador do seu objectivo em puzzles com muitos espaços livres.

### Distância euclidiana do arrumador ao alvo mais próximo

método: `heur_euclidean_usher_box(nodo)`

A segunda heuristica programada foi a distância euclidiana do arrumador à caixa mais próxima. Esta heuristica permite também aproximar o arrumador das caixas quando começa longe das caixas.

### Algoritmo húngaro com distância de manhattan

método: `hung_alg_manh(nodo)`

Considere-se $n$ caixas e $m$ alvos. No jogo sokoban, $n=m$ para qualquer puzzle. 

Sendo que a cada elemento de $n$ pode apenas corresponder um e um só elemento de $m$, apesar de $n_i$ poder corresponder a qualquer $m_j$ ( $\forall{n, m}, n_{i} \to m_{j}$), as combinações deste problema formam um grafo bipartido completo $K_{n,m}$, como estudado na cadeira de Grafos e Redes.

Como o interesse é obter uma solução óptima (menor custo) para este grafo chega-se à conclusão que este é um Problema de Afetação, como estudado em Investigação Operacional.

Foi também escolhida uma forma de calcular o custo de cada combinação possível $n_{i} \to m_{j}$, neste caso baseada na distância de manhattan entre a caixa e o alvo.

Nesta heurística foi usado o algoritmo hungáro (estudado, também, no âmbito da cadeira de Investigação Operacional) para associar uma caixa a um alvo baseado na melhor combinação de custos (minimizar o custo global de $K_{n,m}$).

A complexidade temporal deste algoritmo é de $O(n^3)$, o que é bastante pesado, mas visto que os puzzles dados têm todos $n ≤ 3$ vai ser possivel aplicar este algoritmo em tempo útil, como vamos testar na proxima secção.
Visto que implementar o algoritmo hungáro sai fora do âmbito da cadeira, foi utilizada a implementação [1]: https://github.com/bmc/munkres.

Todo o código necessário está disponivel no ficheiro hungarian.py e está sobre a licença Apache, permitindo o seu uso e modificação.

### Combinação das heuristicas anteriores

Foram também programadas algumas funções que combinam as anteriores:

húngaro + distância eucl. a caixas: `hung_alg_manh_usher_to_box(nodo):`

húngaro + distância eucl. a alvos: `hung_alg_manh_usher_to_target(nodo):`


## Exemplos de execução

No ficheiro `sokoban.py` estão as classes principais para a execução do puzzle. Todo o código de análise de execução apresentado neste relatório está no ficheiro `run-sokoban.py`.

In [ ]:
from sokoban import *

## Análise dos algoritmos experimentados

Vão ser utilizados os puzzles entregues no enunciado para testar os algoritmos disponíveis no ficheiro `search.py` do repositório aima-python, disponibilizado nas aulas. Para além desses, foram construídos mais 7 puzzles para aumentar a amostra estudada.

A função `statistics` (definida em `run-sokoban`) é um método para imprimir dados da resolução de um problema Sokoban.

In [ ]:
def statistics(resultado, caminho=False):
    path = resultado.path()
    solucao = resultado.solution()
    number_moves = 0
    number_pushes = 0

    for index, action in enumerate(solucao):
        accao, _ = action.split()
        if accao == 'andar':
            number_moves += 1
        else:
            number_pushes += 1

    for index, state in enumerate(path):
        if caminho:
            print(state)
    else:
        print('Número de passos:', index)

    print('Números de moves:', number_moves)
    print('Números de pushes:', number_pushes)

In [3]:
puzzle = "puzzle2.txt"
puzzle_estado = import_sokoban_file("puzzles/" + puzzle)

sokoban = Sokoban(puzzle_estado)

In [4]:
%%timeit 
ucs_resultado = uniform_cost_search(sokoban)

387 ms ± 111 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
ucs_resultado = uniform_cost_search(sokoban)
statistics(ucs_resultado)

Número de passos: 43
Números de moves: 31
Números de pushes: 12


In [5]:
%%timeit
bfs_resultado = breadth_first_search(sokoban)

333 ms ± 55.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [6]:
bfs_resultado = breadth_first_search(sokoban)
statistics(bfs_resultado)

Número de passos: 43
Números de moves: 31
Números de pushes: 12


In [8]:
%%timeit
astar_resultado = astar_search(sokoban, hung_alg_manh)

339 ms ± 14.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [10]:
astar_resultado = astar_search(sokoban, hung_alg_manh)
statistics(astar_resultado)

Número de passos: 43
Números de moves: 31
Números de pushes: 12


In [13]:
%%timeit
greedy_resultado = greedy_best_first_graph_search(sokoban, hung_alg_manh)

76.1 ms ± 3.47 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [14]:
greedy_resultado = greedy_best_first_graph_search(sokoban, hung_alg_manh)
statistics(greedy_resultado)

Número de passos: 47
Números de moves: 35
Números de pushes: 12
